In [74]:
import os
import json
import numpy as np 
import pandas as pd
import polars as pl
import seaborn as sns 
import matplotlib as mpl
import scipy.stats as st
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from tqdm import tqdm
from plotly import tools
import plotly as py
from plotly.offline import iplot
import pyarrow
from sklearn.metrics import mean_squared_error
from Utils.dataproc import *
import warnings
warnings.filterwarnings('ignore')

In [75]:
path = '/Users/egortuzarov/ML2/Hw4/Data/aim-2024-transactions'

# 1) Организация обучения и валидации

In [76]:
df = pd.read_parquet(f'{path}/train.parquet')
df['totals_transactionRevenue'] = df['totals_transactionRevenue'].fillna(0)
df["totals_transactionRevenue"] = df["totals_transactionRevenue"].astype('float')
gdf = df.groupby("fullVisitorId")["totals_transactionRevenue"].sum().reset_index()

In [77]:
df.describe()

,date,visitId,visitNumber,visitStartTime,totals_transactionRevenue
count,6.332100e+05,6.332100e+05,633210.000000,6.332100e+05,6.332100e+05
mean,2.016388e+07,1.480146e+09,2.267662,1.480146e+09,1.587463e+06
std,4.255554e+03,5.748743e+06,9.341880,5.748743e+06,4.546771e+07
min,2.016080e+07,1.470035e+09,1.000000,1.470035e+09,0.000000e+00
25%,2.016100e+07,1.475698e+09,1.000000,1.475698e+09,0.000000e+00
50%,2.016112e+07,1.479686e+09,1.000000,1.479686e+09,0.000000e+00
75%,2.017012e+07,1.484918e+09,1.000000,1.484918e+09,0.000000e+00
max,2.017033e+07,1.491030e+09,364.000000,1.491030e+09,1.785550e+10


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633210 entries, 0 to 633209
Data columns (total 35 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   channelGrouping                               633210 non-null  object 
 1   date                                          633210 non-null  int64  
 2   fullVisitorId                                 633210 non-null  object 
 3   sessionId                                     633210 non-null  object 
 4   visitId                                       633210 non-null  int64  
 5   visitNumber                                   633210 non-null  int64  
 6   visitStartTime                                633210 non-null  int64  
 7   device_browser                                633210 non-null  object 
 8   device_operatingSystem                        633210 non-null  object 
 9   device_isMobile                               63

In [79]:
gdf.describe()

,totals_transactionRevenue
count,5.074010e+05
mean,1.981071e+06
std,6.503592e+07
min,0.000000e+00
25%,0.000000e+00
50%,0.000000e+00
75%,0.000000e+00
max,2.653333e+10


In [80]:
nzr = (gdf["totals_transactionRevenue"]>0).sum()
print("Number of unique customers with non-zero revenue : ", nzr, "and the ratio is : ", nzr / gdf.shape[0])

Number of unique customers with non-zero revenue :  6496 and the ratio is :  0.012802497432996782


In [81]:
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
numeric_cols = df.select_dtypes(include=['int', 'float']).columns.tolist()

In [82]:
categorical_cols

['channelGrouping',
 'fullVisitorId',
 'sessionId',
 'device_browser',
 'device_operatingSystem',
 'device_deviceCategory',
 'geoNetwork_continent',
 'geoNetwork_subContinent',
 'geoNetwork_country',
 'geoNetwork_region',
 'geoNetwork_metro',
 'geoNetwork_city',
 'geoNetwork_networkDomain',
 'totals_hits',
 'totals_pageviews',
 'totals_bounces',
 'totals_newVisits',
 'trafficSource_campaign',
 'trafficSource_source',
 'trafficSource_medium',
 'trafficSource_keyword',
 'trafficSource_isTrueDirect',
 'trafficSource_referralPath',
 'trafficSource_adwordsClickInfo.page',
 'trafficSource_adwordsClickInfo.slot',
 'trafficSource_adwordsClickInfo.gclId',
 'trafficSource_adwordsClickInfo.adNetworkType',
 'trafficSource_adwordsClickInfo.isVideoAd',
 'trafficSource_adContent']

In [83]:
cat = []
num = []

# Devices

In [84]:
device_cols = ['device_browser', 'device_operatingSystem', 'device_deviceCategory']
dev = df[device_cols]
dev_encoded = pd.get_dummies(dev)
correlation_matrix = dev_encoded.corr()

In [85]:
dev

,device_browser,device_operatingSystem,device_deviceCategory
0,Chrome,Windows,desktop
1,Firefox,Macintosh,desktop
2,Chrome,Windows,desktop
3,UC Browser,Linux,desktop
4,Chrome,Android,mobile
...,...,...,...
633205,Chrome,Windows,desktop
633206,Chrome,Android,mobile
633207,Android Webview,Android,mobile
633208,Chrome,Windows,desktop


In [86]:
df['device_browser'] = df['device_browser'].astype(str)
df['device_operatingSystem'] = df['device_operatingSystem'].astype(str)
df['device_deviceCategory'] = df['device_deviceCategory'].astype(str)

In [87]:
for col in device_cols:
    cat.append(col)

In [88]:
correlation_matrix

,device_browser_(not set),device_browser_ADM,device_browser_Amazon Silk,device_browser_Android Browser,device_browser_Android Webview,device_browser_Apple-iPhone7C2,device_browser_BlackBerry,device_browser_CSM Click,device_browser_Chrome,device_browser_Coc Coc,...,device_operatingSystem_OpenBSD,device_operatingSystem_Samsung,device_operatingSystem_SunOS,device_operatingSystem_Windows,device_operatingSystem_Windows Phone,device_operatingSystem_Xbox,device_operatingSystem_iOS,device_deviceCategory_desktop,device_deviceCategory_mobile,device_deviceCategory_tablet
device_browser_(not set),1.000000,-0.000004,-0.000070,-0.000070,-0.000229,-0.000006,-0.000042,-0.000004,-0.004035,-0.000083,...,-0.000005,-0.000046,-0.000004,-0.002311,-0.000109,-0.000025,0.000896,0.000218,-0.000013,-0.000503
device_browser_ADM,-0.000004,1.000000,-0.000032,-0.000031,-0.000102,-0.000003,-0.000019,-0.000002,-0.001804,-0.000037,...,-0.000002,-0.000021,-0.000002,-0.001033,-0.000049,-0.000011,-0.000426,0.000692,-0.000632,-0.000225
device_browser_Amazon Silk,-0.000070,-0.000032,1.000000,-0.000628,-0.002041,-0.000055,-0.000376,-0.000032,-0.036007,-0.000737,...,-0.000045,-0.000410,-0.000032,-0.020624,-0.000971,-0.000223,-0.008501,-0.045519,-0.010256,0.134668
device_browser_Android Browser,-0.000070,-0.000031,-0.000628,1.000000,-0.002039,-0.000055,-0.000375,-0.000031,-0.035961,-0.000736,...,-0.000045,-0.000409,-0.000031,-0.020598,-0.000969,-0.000223,-0.008491,-0.045313,0.046982,0.001701
device_browser_Android Webview,-0.000229,-0.000102,-0.002041,-0.002039,1.000000,-0.000177,-0.001219,-0.000102,-0.116870,-0.002391,...,-0.000145,-0.001330,-0.000102,-0.066940,-0.003150,-0.000723,-0.027593,-0.147746,0.146115,0.021917
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
device_operatingSystem_Xbox,-0.000025,-0.000011,-0.000223,-0.000223,-0.000723,-0.000019,-0.000133,-0.000011,-0.012759,-0.000261,...,-0.000016,-0.000145,-0.000011,-0.007308,-0.000344,1.000000,-0.003012,0.004896,-0.004468,-0.001591
device_operatingSystem_iOS,0.000896,-0.000426,-0.008501,-0.008491,-0.027593,-0.000738,-0.005077,-0.000426,-0.402902,-0.009958,...,-0.000602,-0.005539,-0.000426,-0.278768,-0.013119,-0.003012,1.000000,-0.615070,0.500558,0.340538
device_deviceCategory_desktop,0.000218,0.000692,-0.045519,-0.045313,-0.147746,0.001199,-0.027184,0.000692,0.183247,0.015421,...,0.000979,-0.029656,0.000692,0.443974,-0.070245,0.004896,-0.615070,1.000000,-0.912685,-0.324864
device_deviceCategory_mobile,-0.000013,-0.000632,-0.010256,0.046982,0.146115,-0.001095,0.029785,-0.000632,-0.138857,-0.014772,...,-0.000894,0.032494,-0.000632,-0.406209,0.076864,-0.004468,0.500558,-0.912685,1.000000,-0.089999


In [89]:
def na_per(df, name):
    nan_fraction = df.isna().mean()
    print(f"Доля NaN в колонке {name}:")
    print(nan_fraction)

In [90]:
for col in device_cols:
    na_per(df[col], col)

Доля NaN в колонке device_browser:
0.0
Доля NaN в колонке device_operatingSystem:
0.0
Доля NaN в колонке device_deviceCategory:
0.0


In [91]:
def horizontal_bar_chart(cnt_srs, color):
    trace = go.Bar(
        y=cnt_srs.index[::-1],
        x=cnt_srs.values[::-1],
        showlegend=False,
        orientation = 'h',
        marker=dict(
            color=color,
        ),
    )
    return trace

# Device Browser
cnt_srs = df.groupby('device_browser')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')

# Device Category
cnt_srs = df.groupby('device_deviceCategory')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(71, 58, 131, 0.8)')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(71, 58, 131, 0.8)')
trace6 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(71, 58, 131, 0.8)')

# Operating system
cnt_srs = df.groupby('device_operatingSystem')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(246, 78, 139, 0.6)')
trace8 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10),'rgba(246, 78, 139, 0.6)')
trace9 = horizontal_bar_chart(cnt_srs["mean"].head(10),'rgba(246, 78, 139, 0.6)')

# Creating two subplots
fig = tools.make_subplots(rows=3, cols=3, vertical_spacing=0.04, 
                          subplot_titles=["Device Browser - Count", "Device Browser - Non-zero Revenue Count", "Device Browser - Mean Revenue",
                                          "Device Category - Count",  "Device Category - Non-zero Revenue Count", "Device Category - Mean Revenue", 
                                          "Device OS - Count", "Device OS - Non-zero Revenue Count", "Device OS - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)

fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="Device Plots")
iplot(fig, filename='device-plots')

# DateTime

In [92]:
import datetime

def scatter_plot(cnt_srs, color):
    trace = go.Scatter(
        x=cnt_srs.index[::-1],
        y=cnt_srs.values[::-1],
        showlegend=False,
        marker=dict(
            color=color,
        ),
    )
    return trace

df['date'] = df['date'].apply(lambda x: datetime.date(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:])))
cnt_srs = df.groupby('date')['totals_transactionRevenue'].agg(['size', 'count'])
cnt_srs.columns = ["count", "count of non-zero revenue"]
cnt_srs = cnt_srs.sort_index()
#cnt_srs.index = cnt_srs.index.astype('str')
trace1 = scatter_plot(cnt_srs["count"], 'red')
trace2 = scatter_plot(cnt_srs["count of non-zero revenue"], 'blue')

fig = tools.make_subplots(rows=2, cols=1, vertical_spacing=0.08,
                          subplot_titles=["Date - Count", "Date - Non-zero Revenue count"])
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout'].update(height=800, width=800, paper_bgcolor='rgb(233,233,233)', title="Date Plots")
iplot(fig, filename='date-plots')

# Geography

In [93]:
geo_cols = ['geoNetwork_continent',
 'geoNetwork_subContinent',
 'geoNetwork_country',
 'geoNetwork_region',
 'geoNetwork_metro',
 'geoNetwork_city',
 'geoNetwork_networkDomain']

geo = df[geo_cols]

In [94]:
for col in geo_cols:
    na_per(df[col], col)

Доля NaN в колонке geoNetwork_continent:
0.0
Доля NaN в колонке geoNetwork_subContinent:
0.0
Доля NaN в колонке geoNetwork_country:
0.0
Доля NaN в колонке geoNetwork_region:
0.0
Доля NaN в колонке geoNetwork_metro:
0.0
Доля NaN в колонке geoNetwork_city:
0.0
Доля NaN в колонке geoNetwork_networkDomain:
0.0


In [95]:
for col in geo_cols:
    cat.append(col)

In [96]:
for col in geo_cols:
    df[col] = df[col].astype(str)

In [97]:
geo

,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_networkDomain
0,Asia,Western Asia,Turkey,Izmir,(not set),Izmir,ttnet.com.tr
1,Oceania,Australasia,Australia,not available in demo dataset,not available in demo dataset,not available in demo dataset,dodo.net.au
2,Europe,Southern Europe,Spain,Community of Madrid,(not set),Madrid,unknown.unknown
3,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown
4,Europe,Northern Europe,United Kingdom,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown
...,...,...,...,...,...,...,...
633205,Americas,Caribbean,Puerto Rico,not available in demo dataset,not available in demo dataset,not available in demo dataset,prtc.net
633206,Asia,Southern Asia,Sri Lanka,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown
633207,Asia,Eastern Asia,South Korea,Seoul,(not set),Seoul,unknown.unknown
633208,Asia,Southeast Asia,Indonesia,not available in demo dataset,not available in demo dataset,not available in demo dataset,unknown.unknown


In [98]:
# Continent
cnt_srs = df.groupby('geoNetwork_continent')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(58, 71, 80, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(58, 71, 80, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(58, 71, 80, 0.6)')

# Sub-continent
cnt_srs = df.groupby('geoNetwork_subContinent')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"], 'orange')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'orange')
trace6 = horizontal_bar_chart(cnt_srs["mean"], 'orange')

# Country
cnt_srs = df.groupby('geoNetwork_country')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace7 = horizontal_bar_chart(cnt_srs["count"], 'red')
trace8 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'red')
trace9 = horizontal_bar_chart(cnt_srs["mean"], 'red')

# Region
cnt_srs = df.groupby('geoNetwork_region')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace10 = horizontal_bar_chart(cnt_srs["count"], 'green')
trace11 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'green')
trace12 = horizontal_bar_chart(cnt_srs["mean"], 'green')

# City
cnt_srs = df.groupby('geoNetwork_city')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace13 = horizontal_bar_chart(cnt_srs["count"], 'yellow')
trace14 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'yellow')
trace15 = horizontal_bar_chart(cnt_srs["mean"], 'yellow')

# Network domain
cnt_srs = df.groupby('geoNetwork_networkDomain')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace16 = horizontal_bar_chart(cnt_srs["count"].head(10), 'blue')
trace17= horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'blue')
trace18 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'blue')

# Creating two subplots
fig = tools.make_subplots(rows=6, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Continent - Count", "Continent - Non-zero Revenue Count", "Continent - Mean Revenue",
                                          "Sub Continent - Count",  "Sub Continent - Non-zero Revenue Count", "Sub Continent - Mean Revenue",
                                          "Country - Count",  "Country - Non-zero Revenue Count", "Country - Mean Revenue",
                                          "Region - Count",  "Region - Non-zero Revenue Count", "Region - Mean Revenue",
                                          "City - Count",  "City - Non-zero Revenue Count", "City - Mean Revenue",
                                          "Network Domain - Count", "Network Domain - Non-zero Revenue Count", "Network Domain - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)
fig.append_trace(trace7, 3, 1)
fig.append_trace(trace8, 3, 2)
fig.append_trace(trace9, 3, 3)
fig.append_trace(trace10, 4, 1)
fig.append_trace(trace11, 4, 2)
fig.append_trace(trace12, 4, 3)
fig.append_trace(trace13, 5, 1)
fig.append_trace(trace14, 5, 2)
fig.append_trace(trace15, 5, 3)
fig.append_trace(trace16, 6, 1)
fig.append_trace(trace17, 6, 2)
fig.append_trace(trace18, 6, 3)

fig['layout'].update(height=1500, width=1200, paper_bgcolor='rgb(233,233,233)', title="Geography Plots")
iplot(fig, filename='geo-plots')

# Total

In [99]:
tot_cols = ['totals_hits',
 'totals_pageviews',
 'totals_bounces',
 'totals_newVisits']
tot = df[tot_cols]

In [100]:
tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633210 entries, 0 to 633209
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   totals_hits       633210 non-null  object
 1   totals_pageviews  633148 non-null  object
 2   totals_bounces    315541 non-null  object
 3   totals_newVisits  498095 non-null  object
dtypes: object(4)
memory usage: 19.3+ MB


In [101]:
tot

,totals_hits,totals_pageviews,totals_bounces,totals_newVisits
0,1,1,1,1
1,1,1,1,1
2,1,1,1,1
3,1,1,1,1
4,1,1,1,None
...,...,...,...,...
633205,17,15,None,1
633206,18,13,None,1
633207,24,21,None,1
633208,24,22,None,1


In [102]:
df['totals_bounces'].unique()

array(['1', None], dtype=object)

In [103]:
df['totals_newVisits'].unique()

array(['1', None], dtype=object)

In [104]:
for col in tot_cols:
    na_per(df[col], col)

Доля NaN в колонке totals_hits:
0.0
Доля NaN в колонке totals_pageviews:
9.791380426714676e-05
Доля NaN в колонке totals_bounces:
0.5016803272216168
Доля NaN в колонке totals_newVisits:
0.21338102683154087


In [105]:
for col in tot_cols:
    df[col].fillna(0, inplace=True)

In [106]:
df['totals_hits'] = df['totals_hits'].astype(int)
df['totals_pageviews'] = df['totals_pageviews'].astype(int)
df['totals_bounces'] = df['totals_bounces'].astype(bool)
df['totals_newVisits'] = df['totals_newVisits'].astype(bool)

In [107]:
for col in tot_cols:
    num.append(col)

In [108]:
cnt_srs = df.groupby('totals_hits')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'green')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'green')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'green')

# Sub-continent
cnt_srs = df.groupby('totals_pageviews')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"], 'purple')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'purple')
trace6 = horizontal_bar_chart(cnt_srs["mean"], 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Traffic Source - Count", "Traffic Source - Non-zero Revenue Count", "Traffic Source - Mean Revenue",
                                          "Traffic Source Medium - Count",  "Traffic Source Medium - Non-zero Revenue Count", "Traffic Source Medium - Mean Revenue"
                                          ])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1000, width=1200, paper_bgcolor='rgb(233,233,233)', title="Totals Plots")
iplot(fig, filename='totals-plots')

# Traffic

In [109]:
traffic_cols = ['trafficSource_campaign',
 'trafficSource_source',
 'trafficSource_medium',
 'trafficSource_keyword',
 'trafficSource_isTrueDirect',
 'trafficSource_referralPath',
 'trafficSource_adwordsClickInfo.page',
 'trafficSource_adwordsClickInfo.slot',
 'trafficSource_adwordsClickInfo.gclId',
 'trafficSource_adwordsClickInfo.adNetworkType',
 'trafficSource_adwordsClickInfo.isVideoAd',
 'trafficSource_adContent']

In [110]:
for col in traffic_cols:
    na_per(df[col], col)

Доля NaN в колонке trafficSource_campaign:
0.0
Доля NaN в колонке trafficSource_source:
0.0
Доля NaN в колонке trafficSource_medium:
0.0
Доля NaN в колонке trafficSource_keyword:
0.597831683011955
Доля NaN в колонке trafficSource_isTrueDirect:
0.7156804219769113
Доля NaN в колонке trafficSource_referralPath:
0.5761800982296552
Доля NaN в колонке trafficSource_adwordsClickInfo.page:
0.9786453151403168
Доля NaN в колонке trafficSource_adwordsClickInfo.slot:
0.9786453151403168
Доля NaN в колонке trafficSource_adwordsClickInfo.gclId:
0.9785679316498476
Доля NaN в колонке trafficSource_adwordsClickInfo.adNetworkType:
0.9786453151403168
Доля NaN в колонке trafficSource_adwordsClickInfo.isVideoAd:
0.9786453151403168
Доля NaN в колонке trafficSource_adContent:
0.991675747382385


In [111]:
tr_colls = ['trafficSource_campaign', 'trafficSource_source', 'trafficSource_medium', 'trafficSource_keyword', 'trafficSource_referralPath']

In [112]:
tr

,trafficSource_campaign,trafficSource_source,trafficSource_medium,trafficSource_keyword,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent
0,(not set),google,organic,(not provided),None,None,None,None,None,None,None,None
1,(not set),google,organic,(not provided),None,None,None,None,None,None,None,None
2,(not set),google,organic,(not provided),None,None,None,None,None,None,None,None
3,(not set),google,organic,google + online,None,None,None,None,None,None,None,None
4,(not set),google,organic,(not provided),True,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
633205,(not set),youtube.com,referral,None,None,/yt/about/,None,None,None,None,None,None
633206,(not set),youtube.com,referral,None,None,/yt/about/,None,None,None,None,None,None
633207,(not set),youtube.com,referral,None,None,/yt/about/ko/,None,None,None,None,None,None
633208,(not set),facebook.com,referral,None,None,/l.php,None,None,None,None,None,None


In [113]:
for col in tr_colls:
    df[col] = df[col].fillna('miss', inplace = True)
    df[col] = df[col].astype(str)
    cat.append(col)


In [114]:
tr = df[traffic_cols]
tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 633210 entries, 0 to 633209
Data columns (total 12 columns):
 #   Column                                        Non-Null Count   Dtype 
---  ------                                        --------------   ----- 
 0   trafficSource_campaign                        633210 non-null  object
 1   trafficSource_source                          633210 non-null  object
 2   trafficSource_medium                          633210 non-null  object
 3   trafficSource_keyword                         633210 non-null  object
 4   trafficSource_isTrueDirect                    180034 non-null  object
 5   trafficSource_referralPath                    633210 non-null  object
 6   trafficSource_adwordsClickInfo.page           13522 non-null   object
 7   trafficSource_adwordsClickInfo.slot           13522 non-null   object
 8   trafficSource_adwordsClickInfo.gclId          13571 non-null   object
 9   trafficSource_adwordsClickInfo.adNetworkType  13522 non-nul

In [123]:
cat

['device_browser',
 'device_operatingSystem',
 'device_deviceCategory',
 'geoNetwork_continent',
 'geoNetwork_subContinent',
 'geoNetwork_country',
 'geoNetwork_region',
 'geoNetwork_metro',
 'geoNetwork_city',
 'geoNetwork_networkDomain',
 'trafficSource_campaign',
 'trafficSource_source',
 'trafficSource_medium',
 'trafficSource_keyword',
 'trafficSource_referralPath']

In [124]:
num

['totals_hits', 'totals_pageviews', 'totals_bounces', 'totals_newVisits']

In [115]:
# Continent
cnt_srs = df.groupby('trafficSource_source')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'green')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'green')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'green')

# Sub-continent
cnt_srs = df.groupby('trafficSource_medium')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace4 = horizontal_bar_chart(cnt_srs["count"], 'purple')
trace5 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"], 'purple')
trace6 = horizontal_bar_chart(cnt_srs["mean"], 'purple')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Traffic Source - Count", "Traffic Source - Non-zero Revenue Count", "Traffic Source - Mean Revenue",
                                          "Traffic Source Medium - Count",  "Traffic Source Medium - Non-zero Revenue Count", "Traffic Source Medium - Mean Revenue"
                                          ])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 2, 1)
fig.append_trace(trace5, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1000, width=1200, paper_bgcolor='rgb(233,233,233)', title="Traffic Source Plots")
iplot(fig, filename='traffic-source-plots')

In [116]:
# Page views
cnt_srs = df.groupby('totals_pageviews')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(60), 'cyan')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(60), 'cyan')
trace5 = horizontal_bar_chart(cnt_srs["mean"].head(60), 'cyan')

# Hits
cnt_srs = df.groupby('totals_hits')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", 'mean']
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace3 = horizontal_bar_chart(cnt_srs["count"].head(60), 'black')
trace4 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(60), 'black')
trace6 = horizontal_bar_chart(cnt_srs["mean"].head(60), 'black')

# Creating two subplots
fig = tools.make_subplots(rows=2, cols=3, vertical_spacing=0.08, horizontal_spacing=0.15, 
                          subplot_titles=["Total Pageviews - Count", "Total Pageviews - Non-zero Revenue Count", "Total Pageviews - Mean Revenue",
                                          "Total Hits - Count",  "Total Hits - Non-zero Revenue Count", "Total Hits - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace5, 1, 3)
fig.append_trace(trace3, 2, 1)
fig.append_trace(trace4, 2, 2)
fig.append_trace(trace6, 2, 3)

fig['layout'].update(height=1200, width=900, paper_bgcolor='rgb(233,233,233)', title="Visitor Profile Plots")
iplot(fig, filename='visitor-profile-plots')

In [117]:
categorical_cols

['channelGrouping',
 'fullVisitorId',
 'sessionId',
 'device_browser',
 'device_operatingSystem',
 'device_deviceCategory',
 'geoNetwork_continent',
 'geoNetwork_subContinent',
 'geoNetwork_country',
 'geoNetwork_region',
 'geoNetwork_metro',
 'geoNetwork_city',
 'geoNetwork_networkDomain',
 'totals_hits',
 'totals_pageviews',
 'totals_bounces',
 'totals_newVisits',
 'trafficSource_campaign',
 'trafficSource_source',
 'trafficSource_medium',
 'trafficSource_keyword',
 'trafficSource_isTrueDirect',
 'trafficSource_referralPath',
 'trafficSource_adwordsClickInfo.page',
 'trafficSource_adwordsClickInfo.slot',
 'trafficSource_adwordsClickInfo.gclId',
 'trafficSource_adwordsClickInfo.adNetworkType',
 'trafficSource_adwordsClickInfo.isVideoAd',
 'trafficSource_adContent']

In [118]:
df[numeric_cols]

,date,visitId,visitNumber,visitStartTime,totals_transactionRevenue
0,2016-09-02,1472830385,1,1472830385,0.0
1,2016-09-02,1472880147,1,1472880147,0.0
2,2016-09-02,1472865386,1,1472865386,0.0
3,2016-09-02,1472881213,1,1472881213,0.0
4,2016-09-02,1472822600,2,1472822600,0.0
...,...,...,...,...,...
633205,2017-01-04,1483554750,1,1483554750,0.0
633206,2017-01-04,1483543798,1,1483543798,0.0
633207,2017-01-04,1483526434,1,1483526434,0.0
633208,2017-01-04,1483592857,1,1483592864,0.0


In [119]:
cnt_srs = df.groupby('visitNumber')['totals_transactionRevenue'].agg(['size', 'count', 'mean'])
cnt_srs.columns = ["count", "count of non-zero revenue", "mean"]
cnt_srs = cnt_srs.sort_values(by="count", ascending=False)
trace1 = horizontal_bar_chart(cnt_srs["count"].head(10), 'rgba(50, 171, 96, 0.6)')
trace2 = horizontal_bar_chart(cnt_srs["count of non-zero revenue"].head(10), 'rgba(50, 171, 96, 0.6)')
trace3 = horizontal_bar_chart(cnt_srs["mean"].head(10), 'rgba(50, 171, 96, 0.6)')

# Creating two subplots
fig = tools.make_subplots(rows=1, cols=3, vertical_spacing=0.04, 
                          subplot_titles=["Visit Number - Count", "Visit Number - Non-zero Revenue Count", "Visit Number - Mean Revenue"])

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)


fig['layout'].update(height=1200, width=1200, paper_bgcolor='rgb(233,233,233)', title="visitNumber Plots")
iplot(fig, filename='visitNumber-plots')
